<a href="https://colab.research.google.com/github/Chutki319239/msazure/blob/main/TWITTER_SENTIMENT_ANALYSIS_(NLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing kaggle library
!pip install kaggle

# **Uploading Kaggle.json file**

In [2]:
# Configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# **Importing Twitter Sentiment Dataset**

In [3]:
# API to fetch the data from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 91% 74.0M/80.9M [00:00<00:00, 163MB/s]
100% 80.9M/80.9M [00:00<00:00, 153MB/s]


In [7]:
# Extracting from the compressed dataset

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


# **Importing the Dependencies**

In [9]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
# Printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# **Data Processing**

In [12]:
# Loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv' , encoding = 'ISO-8859-1')

In [13]:
# Checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [14]:
# Printing the first five rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [15]:
# Naming the columns and reading the dataset again
column_names = ['target','id','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv' , names = column_names ,  encoding = 'ISO-8859-1')

In [16]:
# Checking the number of rows and columns
twitter_data.shape

(1600000, 6)

In [17]:
# Printing the first five rows of the dataframe
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [18]:
# Counting the number of missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [19]:
# Checking the distribution of the target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


# **Convert the target "4" to "1"**

In [21]:
twitter_data.replace({'target': {4: 1}}, inplace=True)

In [22]:
# Checking the distribution of the target column
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


 **0 ---> Negative Tweet , 1 ---> Positive Tweet**

# **Stemming**

Stemming is the process of reducing a word to its Root word ;
example --> actor , actress , acting : act

In [23]:
port_stem = PorterStemmer()

In [26]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [27]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [28]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [29]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [30]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [31]:
# Seperating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [32]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [33]:
print(Y)

[0 0 0 ... 1 1 1]


# **Splitting the data into training and testing data**

In [34]:
X_test , X_train , Y_test , Y_train = train_test_split(X , Y , test_size=0.2 , stratify = Y , random_state = 2)


In [35]:
print(X.shape , X_train.shape , X_test.shape)

(1600000,) (320000,) (1280000,)


In [36]:
print(X_train)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [37]:
print(X_test)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [38]:
# Converting the textual data to numerical data
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [39]:
print(X_train)

  (0, 104606)	0.4459147041194029
  (0, 51603)	0.25774294336856257
  (0, 107797)	0.17983918796121573
  (0, 158685)	0.3174206399662331
  (0, 26316)	0.2701195948137176
  (0, 163496)	0.180932348330391
  (0, 67046)	0.28383380524479984
  (0, 11837)	0.16338171192787918
  (0, 150603)	0.2214371376881977
  (0, 5650)	0.17295837060294847
  (0, 154815)	0.3501587145032445
  (0, 41330)	0.3621075009813407
  (0, 53904)	0.23662705066222536
  (1, 5650)	0.181974929607951
  (1, 2443)	0.2650067441796073
  (1, 99057)	0.2461083000957446
  (1, 141842)	0.21193862113578937
  (1, 134752)	0.4128221547743352
  (1, 83954)	0.35024490258566765
  (1, 56881)	0.48516752160641513
  (1, 136317)	0.5125311002730158
  (2, 71650)	0.5621198218335112
  (2, 99080)	0.2616858867134709
  (2, 13273)	0.3887964554215523
  (2, 8454)	0.35890991973427855
  :	:
  (319994, 65393)	0.4918118278116105
  (319995, 163496)	0.22695940922040267
  (319995, 161549)	0.23733271775016032
  (319995, 122091)	0.28395414359302695
  (319995, 89801)	0.2574670

In [40]:
print(X_test)

  (0, 42634)	0.37277280423207615
  (0, 72344)	0.5336330797330684
  (0, 90675)	0.4219978835085155
  (0, 137247)	0.3578694854673394
  (0, 169740)	0.27246324011939344
  (0, 172332)	0.4425914290955845
  (1, 62896)	1.0
  (2, 30368)	0.3156109452469487
  (2, 42634)	0.45486610017704826
  (2, 48426)	0.18846264603903023
  (2, 50467)	0.28900701382558186
  (2, 51585)	0.20150931378777845
  (2, 58947)	0.18821952017288582
  (2, 69502)	0.1611869934399326
  (2, 102983)	0.24030158624165557
  (2, 111405)	0.16757521808028886
  (2, 157662)	0.32078695921511685
  (2, 158000)	0.1876765098296405
  (2, 158685)	0.15181999980261157
  (2, 168454)	0.3297584531634099
  (2, 172456)	0.3415940567458438
  (3, 21885)	0.522410328719333
  (3, 59392)	0.27816423254687533
  (3, 62145)	0.44468616234250635
  (3, 67476)	0.373017355901726
  :	:
  (1279996, 91817)	0.6216978642678127
  (1279996, 112361)	0.2202889408206576
  (1279996, 123104)	0.2588982467230118
  (1279996, 144509)	0.43068224310653375
  (1279996, 151339)	0.2702228160

# **Training the machine learning model : Logistic Regression**

In [41]:
model = LogisticRegression(max_iter=1000)

In [42]:
model.fit(X_train , Y_train)

LogisticRegression(max_iter=1000)

# **Model Evaluation : Accuracy Score**

In [43]:
# Accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction , Y_train)

In [44]:
print('Accuracy score on the training data: ',training_data_accuracy)

Accuracy score on the training data:  0.810340625


In [45]:
# Accuracy score on the testing data
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction , Y_test)

In [46]:
print('Accuracy score on the testing data: ',testing_data_accuracy)

Accuracy score on the testing data:  0.7703203125


# **Saving the trained model**

In [47]:
import pickle

In [48]:
filename = 'trained_model.sav'
pickle.dump(model , open(filename , 'wb'))

# **Using the saved model for predictions**

In [55]:
# Loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav' , 'rb'))

In [56]:
X_new = X_test[267]
print(Y_test[267])
prediction = model.predict(X_new)
print(prediction)
if(prediction[0]==0):
  print('Negative Tweet')
else:
    print('Positive Tweet')



1
[1]
Positive Tweet


In [51]:
X_new = X_test[3]
print(Y_test[3])
prediction = model.predict(X_new)
print(prediction)
if(prediction[0]==0):
  print('Negative Tweet')
else:
    print('Positive Tweet')

0
[0]
Negative Tweet
